In [1]:
%pip install ../../

Processing /home/hyang/SpuCo
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=81027 sha256=fc291a20ac9a9b3e0005333c549268fe75840752c46d50ecdcf42aa1c062cd6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-2vjr0gjn/wheels/16/2e/00/5bdefcfd7f850d6f19880ecdb4dd3f325f4b906bf004cd82e3
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.utils import set_seed

set_seed(0)

/home/hyang/SpuCo/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from wilds import get_dataset
import torchvision.transforms as transforms

# Load the full dataset, and download it if necessary
dataset = get_dataset(dataset="waterbirds", download=True)

transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])

# Get the training set
train_data = dataset.get_subset(
    "train",
    transform=transform
)

# Get the training set
test_data = dataset.get_subset(
    "test",
    transform=transform
)

In [6]:
val_data = dataset.get_subset(
    "val",
    transform=transform
)

In [7]:
from spuco.utils import WILDSDatasetWrapper

trainset = WILDSDatasetWrapper(dataset=train_data, metadata_spurious_label="background", verbose=True)
testset = WILDSDatasetWrapper(dataset=test_data, metadata_spurious_label="background", verbose=True)
valset = WILDSDatasetWrapper(dataset=val_data, metadata_spurious_label="background", verbose=True)

Partitioning data indices into groups: 100%|██████████| 1199/1199 [00:00<00:00, 1206856.37it/s]


In [ ]:
from spuco.utils import GroupLabeledDataset

group_labeled_trainset = GroupLabeledDataset(trainset, trainset.group_partition)

In [ ]:
from spuco.models import model_factory 

model = model_factory("resnet50", trainset[0][0].shape, trainset.num_classes).to(device)

In [ ]:
from torch.optim import SGD
from spuco.invariant_train import GroupDRO 

group_dro = GroupDRO(
    model=model,
    num_epochs=20,
    trainset=group_labeled_trainset,
    batch_size=128,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9,weight_decay = 1e-4),
    device=device,
    verbose=True
)
group_dro.train()
# erm = ERM(
#     model=model,
#     num_epochs=1,
#     trainset=trainset,
#     batch_size=128,
#     optimizer=SGD(model.parameters(), lr=1e-2, momentum=0.9, nesterov=True),
#     device=device,
#     verbose=True
# )
# erm.train()

Epoch 19: 100%|██████████| 38/38 [00:22<00:00,  1.66batch/s, accuracy=100.0%, loss=0.00181]  


In [ ]:
group_dro.group_weighted_loss.group_weights

tensor([0.2567, 0.2383, 0.2256, 0.2794], device='cuda:6')

In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:  25%|██▌       | 1/4 [00:04<00:12,  4.20s/it]

Group (0, 0) Test Accuracy: 92.72727272727273


Evaluating group-wise accuracy:  50%|█████     | 2/4 [00:08<00:08,  4.19s/it]

Group (0, 1) Test Accuracy: 82.70509977827051


Evaluating group-wise accuracy:  75%|███████▌  | 3/4 [00:10<00:03,  3.06s/it]

Group (1, 0) Test Accuracy: 80.52959501557632


Evaluating group-wise accuracy: 100%|██████████| 4/4 [00:11<00:00,  2.92s/it]

Group (1, 1) Test Accuracy: 88.94080996884735


{(0, 0): 92.72727272727273,
 (0, 1): 82.70509977827051,
 (1, 0): 80.52959501557632,
 (1, 1): 88.94080996884735}